In [11]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import STOPWORDS, WordCloud
from scipy import stats as sts
import statistics as st
from scipy.stats import ks_2samp

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('df_new_factors.csv')
df.head()

,Размер,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Наличие пыльника,Состояние товара,Модель,Винтаж,Наличие коробки,Наличие сертификата,Ценовая категория,Метод производства,Легенда,Цена
0,INT L,Сумки с короткими ручками,MARC JACOBS,Экзотическая кожа,Серебряный,Средние ручки,1,Отличное состояние,Ordinary,0,0,0,Дорогие,Произведенные на фабрике,0,52000
1,INT S,Клатчи и вечерние сумки,VERSACE,Кожа,Черный,Короткие ручки,0,Отличное состояние,Ordinary,0,0,0,Дорогие,Произведенные на фабрике,0,38000
2,INT M,Сумки через плечо,BALENCIAGA,Кожа,Желтый,Длинный ремень,0,Новое с биркой,Crush,0,0,0,Люксовые,Произведенные на фабрике,1,345000
3,INT U,Сумки с короткими ручками,BURBERRY,Кожа,Коричневый,Средние ручки,0,Отличное состояние,Ordinary,1,0,0,Люксовые,Произведенные на фабрике,0,27000
4,INT M,Сумки с короткими ручками,KARL LAGERFELD,Искусственная кожа,Мульти,Средние ручки,0,Новое с биркой,Ordinary,0,0,0,Доступные,Произведенные на фабрике,0,17950


---

### Я выдвигаю ряд гипотез:
1) В среднем люксовые сумки из кожи и ткани стоят одинаково;    
2) Не менее 40% сумок Hermes являются легендарными моделями бренда;        
3) Сумки без пыльника, в среднем стоят дешевле на 30% и более;    
4) Пыльник присутствует у 40% сумок категории "доступный люкс" и более;     
5) Среди сумок класса люкс сумки из бутика и сумки в отличном состоянии стоят всреднем одинаково;     
6) Люксовые сумки без пыльника, в среднем стоят дешевле на 40% и более ;   
7) Не менее 90% сумок Hermes имеет сертификат подлинности;     
8) Сумки Hermes без сертефиката подлинности в среднем стоят дешевле на 60% и более;
9) Люксовые сумки легендарных моделей без сертефиката подлинности в среднем стоят дешевле на 40% и более.

---

## Гипотеза 1
#### В среднем люксовые сумки из кожи и ткани стоят одинаково

Для определения нормальности распределения выборок теперь воспользуемся ***тестом Колмогорова-Смирнова*** 

**Тест Колмогорова-Смирнова**


$$
\begin{aligned}
& H_0: Данные \ выборок \ имеют \ нормальное \ распределение \\
& H_1: Данные \ выборок \ НЕ \ имеют \ нормальное \ распределение
\end{aligned}
$$

In [12]:
luxury = df[df['Ценовая категория'] == 'Люксовые']

leather_lux = luxury[luxury['Материал сумок'] == 'Кожа']['Цена']
textile_lux = luxury[luxury['Материал сумок'] == 'Ткань']['Цена']
ks_2samp(leather_lux, textile_lux)

KstestResult(statistic=0.16573248407643312, pvalue=0.493391537916002, statistic_location=81000, statistic_sign=-1)

Мы видим, что у нас достаточно доказательств для подтверждения нулевой гипотезы — *данные обеих выборок имеют нормальное распределение* (p-value больше 0.05).

**Двухвыборочный t-критерий**

In [13]:
import scipy. stats as stats 
stats. ttest_ind (a=leather_lux, b=textile_lux) 

TtestResult(statistic=1.1584169355136136, pvalue=0.24751418256129423, df=337.0)

p-value > 0,05

__Вывод:__ Гипотеза о том, что среднем люксовые сумки из кожи и ткани стоят одинаково, **не** отвергается на данном уровне значимости.

---

## Гипотеза 2
#### Не менее 40% сумок Hermes являются легендарными моделями бренда

В данном случае мы имеем дело с долями. Пусть $p$ - вероятность того, что сумка Hermes легендарной модели.

Тогда $H_{0}: p = 0.5,$ $H_{1}: p < 0.5 \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\hat {p} - p} {\sqrt {\frac {\hat {p} (1 - \hat {p})} {n}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = -z_{0.95}.$


In [28]:
hermes = df[df['Бренд'] == 'HERMES PRE-OWNED']
n = hermes.shape[0]
p_hat = hermes['Легенда'].mean()

z_obs = (p_hat - 0.4) / (np.sqrt(p_hat * (1 - p_hat) / n))
z_crit = -sts.norm.ppf(0.95)
print(z_obs, z_crit)

z_obs > z_crit

-0.15141706636077498 -1.6448536269514722


True

__Вывод:__ Гипотеза о том, что не менее 40% сумок Hermes являются легендарными моделями бренда, **не** отвергается на данном уровне значимости.

---

## Гипотеза 3
#### Сумки с пыльником, в среднем стоят дороже на 30% и более

Здесь рассмотрим две выборки, независимые по условию: $X$ - цена сумок без пыльника, $Y$ - цена сумок с пыльником.

Тогда $H_{0}: \mu_{x} = 0.7 \mu_{Y}, \space H_{1}: \mu_{X} > 0.7 \mu_{Y} \Rightarrow$ рассмотрим случайную величину $\bar{X} - 0.7 \bar{Y} \overset{asy}{\sim} N(\mu_{Y} - 0.7 \mu_{Y}, \frac {\sigma_{X}^2} {n_{X}} + 0.49 \frac {\sigma_{Y}^2} {n_{Y}}) \overset{H_{0}}{\Rightarrow}$

$\overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \scriptstyle 0.7 \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \scriptstyle 0.49 \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.95}.$

In [25]:

X = df[df['Наличие пыльника'] == 0]['Цена'].values
Y = df[df['Наличие пыльника'] == 1]['Цена'].values

n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

z_obs = (X_mean - 0.7 * Y_mean) / np.sqrt(X_var / n_X + 0.49 * Y_var / n_Y)
z_crit = sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Наблюдаемое значение меньше критического
z_obs < z_crit

-1.6303437920505335 1.6448536269514722


True

__Вывод:__ Гипотеза о том, что сумки с пыльником, в среднем стоят дороже на 30% и более, **не** отвергается на данном уровне значимости.

---

## Гипотеза 4
#### Пыльник присутствует у 40% сумок категории "доступный люкс" и более

В данном случае мы имеем дело с долями. Пусть $p$ - вероятность того, что сумка категории "доступный люкс" имеет пыльник.

Тогда $H_{0}: p = 0.5,$ $H_{1}: p < 0.5 \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\hat {p} - p} {\sqrt {\frac {\hat {p} (1 - \hat {p})} {n}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = -z_{0.95}.$


In [35]:
acessible_luxury = df[df['Ценовая категория'] == 'Доступный люкс']
n = acessible_luxury.shape[0]
p_hat = acessible_luxury['Наличие пыльника'].mean()

z_obs = (p_hat - 0.4) / (np.sqrt(p_hat * (1 - p_hat) / n))
z_crit = -sts.norm.ppf(0.95)
print(z_obs, z_crit)

z_obs > z_crit

-2.836155082112896 -1.6448536269514722


False

**Вывод:** Отвергаем данную гипотезу. Следовательно, наличие или отсутсвие пыльника это скорее особенность ценовой категории, чем фактор влияющий на цену сумки независимо от категории

---

## Гипотеза 5
### Среди сумок класса люкс сумки из бутика и сумки в отличном состоянии стоят всреднем одинаково

**Тест Колмогорова-Смирнова**


$$
\begin{aligned}
& H_0: Данные \ выборок \ имеют \ нормальное \ распределение \\
& H_1: Данные \ выборок \ НЕ \ имеют \ нормальное \ распределение
\end{aligned}
$$

In [37]:
luxury = df[df['Ценовая категория'] == 'Люксовые']

butique_lux = luxury[luxury['Состояние товара'] == 'Новое с биркой']['Цена']
fine_lux = luxury[luxury['Состояние товара'] == 'Отличное состояние']['Цена']
ks_2samp(butique_lux, fine_lux)

KstestResult(statistic=0.3473086408579768, pvalue=9.717253876771703e-12, statistic_location=125000, statistic_sign=-1)

Мы видим, что у нас достаточно доказательств для отвержения нулевой гипотезы — *данные обеих выборок не имеют нормального распределения* (p-value меньше 0.05).

**Критерий Уилкоксона**

Так как нормальность выборок не подтвердилась, а значит предпосылки для использования t-статистики не были выполнены, прибегнем к использованию **Критерия Уилкоксона** (в силу того, что наши выборки зависимы). 

Проверим их объем (<25): 

In [40]:
n = len(butique_lux)
m = len(fine_lux)
print(f'Длина выборки сумок из бутика: {n}')
print(f'Длина выборки сумок в отличном состоянии: {m}')

Длина выборки сумок из бутика: 229
Длина выборки сумок в отличном состоянии: 193


">25", следовательно используем **Z-score**

Для проверки гипотезы о том, что  𝜇
  цен сумок из бутика равно  𝜇
  цен сумок в отличном состоянии, воспользуемся z-статистикой, так как мы не знаем дисперсий наших данных.

Сформулируем гипотезы:

$$
\begin{aligned}
& H_0: \mu_{boutique} = \mu_{fine} \\
& H_1: \mu_{fine} < \mu_{boutique}
\end{aligned}
$$

Уровень значимости $\alpha$ = 0.05 

*Формула z-статистики:*

$$
z = \frac {\bar{X} - \bar{Y} - \mu_0}{\sqrt{\frac {\hat{\sigma}_X^2}{n} + {\frac {\hat{\sigma}_Y^2}{m}}}}
$$

In [46]:
mean_boutique = butique_lux .mean()
mean_fine = fine_lux.mean()
var_boutique = st.variance(butique_lux)
var_fine = st.variance(fine_lux)
z_test = (mean_boutique - mean_fine - 0)/(np.sqrt(var_boutique/n +var_fine/m))
z_test

3.3057975656657628

In [47]:
alpha = 0.05
z_crit = sts.norm.ppf(alpha)
if z_test < z_crit:
    print("H0 отвергается")
else:
    print("Нет оснований отвергать H0")

Нет оснований отвергать H0


**Вывод**: Мы видим, что полученное нами значение больше левостороннего критического значения Z, поэтому результаты теста являются статистически не значимыми. Вот почему у нас нет оснований для того, чтобы отвергнуть нулевую гипотезу, а значит среди сумок класса люкс сумки из бутика и сумки в отличном состоянии стоят всреднем одинаково.

---

## Гипотеза 6
#### Люксовые сумки без пыльника, в среднем стоят дешевле на 40% и более

Здесь рассмотрим две выборки, независимые по условию: $X$ - цена люксовых сумок без пыльника, $Y$ - цена люксовых сумок с пыльником.

Тогда $H_{0}: \mu_{x} = 0.6 \mu_{Y}, \space H_{1}: \mu_{X} > 0.6 \mu_{Y} \Rightarrow$ рассмотрим случайную величину $\bar{X} - 0.6 \bar{Y} \overset{asy}{\sim} N(\mu_{Y} - 0.6 \mu_{Y}, \frac {\sigma_{X}^2} {n_{X}} + 0.36 \frac {\sigma_{Y}^2} {n_{Y}}) \overset{H_{0}}{\Rightarrow}$

$\overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \scriptstyle 0.6 \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \scriptstyle 0.36 \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.95}.$

In [52]:
X = luxury[luxury['Наличие пыльника'] == 0]['Цена'].values
Y = luxury[luxury['Наличие пыльника'] == 1]['Цена'].values

n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

z_obs = (X_mean - 0.6 * Y_mean) / np.sqrt(X_var / n_X + 0.36 * Y_var / n_Y)
z_crit = sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Наблюдаемое значение меньше критического
z_obs < z_crit

0.6890059346329821 1.6448536269514722


True

__Вывод:__ Гипотеза о том, что люксовые сумки без пыльника, в среднем стоят дешевле на 40% и более, **не** отвергается на данном уровне значимости. Следовательно, для очень дорогих сумок наличие пыльника сильно положительно влияет на цену

---

## Гипотеза 7
### Не менее 90% сумок Hermes имеет сертификат подлинности

В данном случае мы имеем дело с долями. Пусть $p$ - вероятность того, что сумка Hermes имеет сертификат подлинности.

Тогда $H_{0}: p = 0.9,$ $H_{1}: p < 0.9 \overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\hat {p} - p} {\sqrt {\frac {\hat {p} (1 - \hat {p})} {n}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = -z_{0.95}.$

In [53]:
hermes = df[df['Бренд'] == 'HERMES PRE-OWNED']
n = hermes.shape[0]
p_hat = hermes['Наличие пыльника'].mean()

z_obs = (p_hat - 0.9) / (np.sqrt(p_hat * (1 - p_hat) / n))
z_crit = -sts.norm.ppf(0.95)
print(z_obs, z_crit)

z_obs > z_crit

-9.123908455454432 -1.6448536269514722


False

__Вывод:__ Гипотеза о том, что не менее 90% сумок Hermes имеет сертификат подлинности, отвергается на данном уровне значимости.

---

## Гипотеза 8
### Сумки Hermes без сертефиката подлинности в среднем стоят дешевле на 60% и более

Здесь рассмотрим две выборки, независимые по условию: $X$ - цена сумок Hermes без сертификата, $Y$ - цена сумок Hermes с сертификатом.

Тогда $H_{0}: \mu_{x} = 0.4 \mu_{Y}, \space H_{1}: \mu_{X} > 0.4 \mu_{Y} \Rightarrow$ рассмотрим случайную величину $\bar{X} - 0.4 \bar{Y} \overset{asy}{\sim} N(\mu_{Y} - 0.4 \mu_{Y}, \frac {\sigma_{X}^2} {n_{X}} + 0.16 \frac {\sigma_{Y}^2} {n_{Y}}) \overset{H_{0}}{\Rightarrow}$

$\overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \scriptstyle 0.4 \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \scriptstyle 0.16 \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.95}.$

In [75]:
X = hermes[hermes['Наличие сертификата'] == 0]['Цена'].values
Y = hermes[hermes['Наличие сертификата'] == 1]['Цена'].values

n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

z_obs = (X_mean - 0.4 * Y_mean) / np.sqrt(X_var / n_X + 0.16 * Y_var / n_Y)
z_crit = sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Наблюдаемое значение меньше критического
z_obs < z_crit

1.5075677964471286 1.6448536269514722


True

__Вывод:__ Гипотеза о том, что сумки Hermes без сертефиката подлинности в среднем стоят дешевле на 60% и более, **не** отвергается на данном уровне значимости.

---

## Гипотеза 9
### Люксовые сумки легендарных моделей без сертефиката подлинности в среднем стоят дешевле на 40% и более

Здесь рассмотрим две выборки, независимые по условию: $X$ - цена люксовых сумок легендарных моделей без сертификата, $Y$ - цена люксовых сумок легендарных моделей с сертификатом.

Тогда $H_{0}: \mu_{x} = 0.6 \mu_{Y}, \space H_{1}: \mu_{X} > 0.6 \mu_{Y} \Rightarrow$ рассмотрим случайную величину $\bar{X} - 0.6 \bar{Y} \overset{asy}{\sim} N(\mu_{Y} - 0.6 \mu_{Y}, \frac {\sigma_{X}^2} {n_{X}} + 0.36 \frac {\sigma_{Y}^2} {n_{Y}}) \overset{H_{0}}{\Rightarrow}$

$\overset{H_{0}}{\Rightarrow} z_{obs} = \frac {\bar{X} - \scriptstyle 0.6 \bar{Y}} {\sqrt{\frac {\hat{\sigma}_{X}^2} {n_{X}} + \scriptstyle 0.36 \frac {\hat{\sigma}_{Y}^2} {n_{Y}}}} \overset{asy}{\sim} N(0, 1), \space z_{crit} = z_{0.95}.$

In [74]:
legendary = luxury[luxury['Легенда'] == 1]
X = legendary[legendary['Наличие сертификата'] == 0]['Цена'].values
Y = legendary[legendary['Наличие сертификата'] == 1]['Цена'].values

n_X = X.shape[0]
n_Y = Y.shape[0]

X_mean = X.mean()
Y_mean = Y.mean()

X_var = X.var(ddof=1)
Y_var = Y.var(ddof=1)

z_obs = (X_mean - 0.6 * Y_mean) / np.sqrt(X_var / n_X + 0.36 * Y_var / n_Y)
z_crit = sts.norm.ppf(0.95)
print(z_obs, z_crit)

# Наблюдаемое значение меньше критического
z_obs < z_crit

0.7973813315539116 1.6448536269514722


True

**Вывод:** Гипотеза о том, что люксовые сумки легендарных моделей без сертефиката подлинности в среднем стоят дешевле на 40% и более, не отвергается на данном уровне значимости. Это означает, что для других легендарных сумок помимо сумок Hermes отсутствие сертефиката подлинности очень сильно сказывается на стоимости.